<a href="https://colab.research.google.com/github/BatalovaEvgeniya/HSE_ML_2018-2019/blob/master/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!pip install xmltodict

In [0]:
def xml_to_df(path):
  import xmltodict
  from xml.parsers.expat import ExpatError
  with open(path) as f:
    data = f.read()
    try:
        doc = xmltodict.parse(data)
    except ExpatError:
        doc = xmltodict.parse(data[3:])
        
  df = pd.DataFrame(list(list(list(doc.values())[0].items())[0][1].values())[1])
  df['report_period_end'] = df['report_period_end'].apply(lambda x: x.split('-')[0] )
  df.value = df.value.astype('int32')
  df = df.drop('report_period_begin', axis=1)
  df = df.groupby(['region_name','report_period_end'])['value'].sum()
  
      
  
  region_year_index = []
  for region in df.index.levels[0]:
    for year in df.xs(region).index:
      region_year_index.append(str(region + ' ' + year))
  df = pd.DataFrame(df.values, index=region_year_index).T
  
  
  
  
  for column_name in df.columns.values:
    if 'Федерация' in column_name or 'федеральный округ' in column_name or 'без авт' in column_name or ' ФО ' in column_name:
      df = df.drop(column_name, axis=1)
      continue
    if '  в том числе:\n' in column_name:
      df = df.rename({column_name: column_name.split('\n')[1]}, axis='columns')
      continue
    if 'в том числе' in column_name:
      temp_split = column_name.split(' ')
      if len(temp_split) > 3:
        df_f= df.rename({column_name: temp_split[3] + ' ' + temp_split[4]}, axis='columns')
      else:
        df_ = df.drop(column_name, axis=1)
      continue
      
    
      
  for column_name in df.columns.values:
    if 'г.' in column_name :
      temp_split = column_name.split(' ')
      if len(temp_split)>1:
        df = df.rename({column_name: temp_split[0] + temp_split[1]}, axis='columns')
    if 'г .' in column_name :
      temp_split = column_name.split(' ')
      if len(temp_split)>1:
        df = df.rename({column_name: temp_split[0] + temp_split[1:]}, axis='columns')
  df_res = df.T

  
  return df_res

In [0]:
def xls_parcer(path, label):
  df_ = pd.read_excel(path, encoding='cp1251')
  df_ = df_.T
  df_.columns = df_.iloc[0]
  df_ = df_.drop(df_.index[0], axis=0)
  for column_name in df_.columns.values:
    if type(column_name) != str:
      continue
    if 'Федерация' in column_name or 'федеральный округ' in column_name or 'без авт' in column_name or ' ФО ' in column_name:
      df_ = df_.drop(column_name, axis=1)
      continue
    if '  в том числе:\n' in column_name:
      df_ = df_.rename({column_name: column_name.split('\n')[1]}, axis='columns')
      continue
    if 'в том числе' in column_name:
      temp_split = column_name.split(' ')
      if len(temp_split) > 3:
        df_ = df_.rename({column_name: temp_split[3] + ' ' + temp_split[4]}, axis='columns')
      else:
        df_ = df_.drop(column_name, axis=1)
      continue
      
      
  for column_name in df_.columns.values:
    if type(column_name) != str:
      continue
    if 'г.' in column_name:
      temp_split = column_name.split(' ')
      if len(temp_split)>1:
        df_ = df_.rename({column_name: temp_split[0] + temp_split[1]}, axis='columns')
    if 'авт.' in column_name:
      temp_split = column_name.split(' ')
      df_ = df_.rename({column_name:temp_split[0] + ' АО' }, axis='columns')
    if 'автономная ' in column_name or 'автономный ' in column_name:
      temp_split = column_name.split(' ')
      df_ = df_.rename({column_name:temp_split[0] + ' АО' }, axis='columns')
    if column_name == ' ':
      try:
        df_ = df_.drop(column_name, axis=1)
      except:
        continue
  cols = []
  for col_ in df_.columns:
    if type(col_)==str:
      cols.append(col_)
  print()
  df_.columns = cols
  region_year_index = []
  for region in df_.columns:
    for year in df_.index:
      region_year_index.append(str(region + ' ' + str(year)))
  df_ = df_.T
  df_res = pd.DataFrame(df_.values.flatten(), index=region_year_index, columns=[str(label)])
  return df_res

In [0]:
def xls_parcer_with_date(path, label, year):
  df_ = pd.read_excel(path, encoding='cp1251', header=1)
  df_ = df_.T
  df_.columns = df_.iloc[0]
  df_ = df_.drop(df_.index[0], axis=0)
  for column_name in df_.columns.values:
    if type(column_name) != str:
      continue
    if 'Федерация' in column_name or 'федеральный округ' in column_name or 'без авт' in column_name or ' ФО ' in column_name:
      df_ = df_.drop(column_name, axis=1)
      continue
    if '  в том числе:\n' in column_name:
      df_ = df_.rename({column_name: column_name.split('\n')[1]}, axis='columns')
      continue
    if 'в том числе' in column_name:
      temp_split = column_name.split(' ')
      if len(temp_split) > 3:
        df_ = df_.rename({column_name: temp_split[3] + ' ' + temp_split[4]}, axis='columns')
      else:
        df_ = df_.drop(column_name, axis=1)
      continue
  for column_name in df_.columns.values:
    if type(column_name) != str:
      continue
    temp_split = column_name.split(' ')
    if 'г.' in column_name:
      if len(temp_split)>1:
        df_ = df_.rename({column_name: temp_split[0] + temp_split[1]}, axis='columns')
    if '(' in column_name:
      temp_split = column_name.split(' (')
      df_ = df_.rename({column_name:temp_split[0]}, axis='columns')
    if 'авт.' in column_name:
      df_ = df_.rename({column_name:temp_split[0] + ' АО' }, axis='columns')
    if 'автономная ' in column_name or 'автономный ' in column_name:
      df_ = df_.rename({column_name:temp_split[0] + ' АО' }, axis='columns')
    if column_name == ' ':
      try:
        df_ = df_.drop(column_name, axis=1)
      except:
        continue
  cols = []
  for col_ in df_.columns:
    if type(col_)==str:
      cols.append(col_)
  df_.columns = cols
  region_year_index = []
  for region in df_.columns:
    region_year_index.append(str(region + ' ' + str(year)))
  
  df_ = df_.T
  
  df_res = pd.DataFrame(df_.values, index=region_year_index, columns=label)
  return df_res

In [0]:
path = 'gdrive/My Drive/Colab Notebooks/crimes/'

In [0]:
df_crimes = xml_to_df(path + 'Всего зарегистрировано преступлений.xml')

In [0]:
df_corp_2018 = xls_parcer_with_date(path + 'demb-dem-pok-sub-2018.xlsx', ['Кол-во актвивных предприятий', 'Кол-во умерших предприятий'] ,2018)
df_invest = xls_parcer(path + 'invest_sub.xlsx', 'Инвестиции')
df_unemployment = xls_parcer(path + 'trud6_15-72.xlsx', 'Уровень безработицы')

In [0]:
year_  = '2018'


ind_inv, value_inv = [], []
for i in range(len(df_invest.index)):
  for j in range(len(df_crimes.index)):
    temp_split = df_invest.index[i].split(' ')
    if (df_invest.index[i] == df_crimes.index[j]) and temp_split[-1] == year_:
      ind_inv.append(df_invest.index[i])
      value_inv.append(df_invest.iloc[i][0])
      continue
      
      
ind_unemp, value_unemp = [], []
for i in range(len(df_unemployment.index)):
  for j in range(len(ind_inv)):
    temp_split = df_unemployment.index[i].split(' ')
    if(df_unemployment.index[i] == ind_inv[j]) and temp_split[-1] == year_:
      ind_unemp.append(df_unemployment.index[i])
      value_unemp.append(df_unemployment.iloc[i][0])
    
ind_crime, value_crime = [], []
for i in range(len(df_crimes.index)):
  for j in range(len(ind_inv)):
    temp_split = df_crimes.index[i].split(' ')
    if (ind_inv[j] == df_crimes.index[i]) and temp_split[-1] == year_:
      ind_crime.append(df_crimes.index[i])
      value_crime.append(df_crimes.iloc[i][0])
      continue

In [0]:
print(len(ind_unemp), len(ind_inv), len(ind_crime))

In [0]:
df_inv = pd.DataFrame(value_inv, index=ind_inv, columns=df_invest.columns)
df_unemp = pd.DataFrame(value_unemp, index=ind_unemp, columns=df_unemployment.columns)
df_cr = pd.DataFrame(value_crime, index=ind_crime, columns=['Уровень преступности'])

In [0]:
df_all = df_inv.merge(df_cr, how='left',left_index=True, right_index=True)
df_all = df_all.merge(df_corp_2018, how='left',left_index=True, right_index=True)
df_all.head()

In [0]:
df_all.isna().sum()

In [0]:
df_all = df_all.dropna()
df_all.isna().sum()

In [0]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df_all.values)

df_scaled = pd.DataFrame(df_scaled, index=df_all.index, columns=df_all.columns)

X, y = df_scaled.drop('Уровень преступности', axis=1), df_scaled['Уровень преступности']

In [0]:
sns.set()
sns.pairplot(df_scaled, size = 3.5);
plt.show();

In [0]:
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.font_manager

from sklearn import svm
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

def OutLierDetection(df,feature1,feature2,outliers_fraction=0.3):
    
    new_df = df.copy()
    rng = np.random.RandomState(42)

    # Example settings
    n_samples = new_df.shape[0]
#     outliers_fraction = 0.2 # ************************************** imp
    clusters_separation = [0]#, 1, 2]

    # define two outlier detection tools to be compared
    classifiers = {
        #"One-Class SVM": svm.OneClassSVM(nu=0.95 * outliers_fraction + 0.05,
           #                              kernel="rbf", gamma=0.1),
        #"Robust covariance": EllipticEnvelope(contamination=outliers_fraction),
        "Isolation Forest": IsolationForest(max_samples=n_samples,
                                            contamination=outliers_fraction,
                                            random_state=rng),
        #"Local Outlier Factor": LocalOutlierFactor(
         #   n_neighbors=35,
            #contamination=outliers_fraction)
    }

    
    xx, yy = np.meshgrid(np.linspace(new_df[feature1].min()-new_df[feature1].min()*10/100, 
                                     new_df[feature1].max()+new_df[feature1].max()*10/100, 50),
                         np.linspace(new_df[feature2].min()-new_df[feature2].min()*10/100,
                                     new_df[feature2].max()+new_df[feature2].max()*10/100, 50))


    n_inliers = int((1. - outliers_fraction) * n_samples)
    n_outliers = int(outliers_fraction * n_samples)
    ground_truth = np.ones(n_samples, dtype=int)
    ground_truth[-n_outliers:] = -1

    # Fit the problem with varying cluster separation
    for i, offset in enumerate(clusters_separation):
        np.random.seed(42)
        # Data generation

        X = new_df[[feature1,feature2]].values.tolist()

        # Fit the model
        plt.figure(figsize=(9, 7))
        for i, (clf_name, clf) in enumerate(classifiers.items()):
            # fit the data and tag outliers
            if clf_name == "Local Outlier Factor":
                y_pred = clf.fit_predict(X)
                scores_pred = clf.negative_outlier_factor_
            else:
                clf.fit(X)
                scores_pred = clf.decision_function(X)
                y_pred = clf.predict(X)
            threshold = stats.scoreatpercentile(scores_pred,
                                                100 * outliers_fraction)
            n_errors = (y_pred != ground_truth).sum()
            
            unique, counts = np.unique(y_pred,return_counts=True)
            print(clf_name,dict(zip(unique, counts)))
            
            new_df[feature1+'_'+feature2+clf_name] = y_pred
#             print(clf_name,y_pred) 
            # plot the levels lines and the points
            if clf_name == "Local Outlier Factor":
                # decision_function is private for LOF
                Z = clf._decision_function(np.c_[xx.ravel(), yy.ravel()])
            else:
                Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
            Z = Z.reshape(xx.shape)
            subplot = plt.subplot(2, 2, i + 1)
            subplot.contourf(xx, yy, Z, levels=np.linspace(Z.min(), threshold, 7),
                             cmap=plt.cm.Blues_r)
            a = subplot.contour(xx, yy, Z, levels=[threshold],
                                linewidths=2, colors='red')
            subplot.contourf(xx, yy, Z, levels=[threshold, Z.max()],
                             colors='orange')
            b = plt.scatter(new_df[feature1], new_df[feature2], c='white',
                     s=20, edgecolor='k')

            subplot.axis('tight')

            subplot.set_xlabel("%s" % (feature1))
 
            plt.ylabel(feature2)#, fontsize=18)
            plt.title("%d. %s (errors: %d)" % (i + 1, clf_name, n_errors))

        plt.subplots_adjust(0.04, 0.1, 0.96, 0.94, 0.1, 0.26)
#         plt.suptitle("Outlier detection")

    plt.show();
    return new_df

In [0]:
df_all.columns

In [0]:
df_scaled = OutLierDetection(df_scaled, 'Инвестиции', 'Кол-во умерших предприятий', outliers_fraction=0.1)

In [0]:
df_scaled = OutLierDetection(df_scaled, 'Уровень преступности', 'Кол-во актвивных предприятий', 0.1)

In [0]:
df_scaled = OutLierDetection(df_scaled, 'Уровень преступности', 'Кол-во умерших предприятий', 0.07)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [0]:
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import  SVR
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor

regressors = [
    svm.SVR(),
    KNeighborsRegressor(n_neighbors=3, algorithm='kd_tree'),
    linear_model.SGDRegressor(),
    linear_model.BayesianRidge(),
    linear_model.LassoLars(),
    linear_model.LinearRegression(),
    linear_model.ElasticNet()]


quality = 1e+30

for item in regressors:
    print(item)
    regr = Pipeline([('poly', PolynomialFeatures(2)),
                     ('scaler', MinMaxScaler(feature_range = (0, 1))),
                    ('linear', item)])
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)
    print('{:10.4e}'.format(mean_squared_error(y_pred, y_test)))
    if (quality - mean_squared_error(y_pred, y_test))>0:
      quality = mean_squared_error(y_pred, y_test)
      model = regr
    
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('best quality', '{:10.4e}'.format(quality))
print('best model', model)

In [0]:
fig = plt.figure(figsize=(10, 12))
plt.plot(y_pred);
plt.plot(y_test)

In [0]:
y_pred